In [ ]:
import pandas as pd
import numpy as np


import string
import re
from pprint import pprint

# NLTK 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english') #this depends on each language

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
np.random.seed(2020)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
df_2009 = pd.read_csv('rajyasabha_questions_and_answers_2009.csv',engine='python')
df_2010 = pd.read_csv('rajyasabha_questions_and_answers_2010.csv',engine='python')
df_2011 = pd.read_csv('rajyasabha_questions_and_answers_2011.csv',engine='python')
df_2012 = pd.read_csv('rajyasabha_questions_and_answers_2012.csv',engine='python')
df_2013 = pd.read_csv('rajyasabha_questions_and_answers_2013.csv',engine='python')
df_2014 = pd.read_csv('rajyasabha_questions_and_answers_2014.csv',engine='python')
df_2015 = pd.read_csv('rajyasabha_questions_and_answers_2015.csv',engine='python')
df_2016 = pd.read_csv('rajyasabha_questions_and_answers_2016.csv',engine='python')
df_2017 = pd.read_csv('rajyasabha_questions_and_answers_2017.csv',engine='python')
df = pd.concat([df_2009, df_2010, df_2011,df_2012,df_2013,df_2014,df_2015,df_2016,df_2017])

In [ ]:
BAD_CHARS = ['(?) ?????']
pat = '|'.join(['({})'.format(re.escape(c)) for c in BAD_CHARS])
df[df['question_description'].str.contains(pat)]

In [ ]:
df_new=df['question_description'].str.split(";", 1, expand=True)
df_new.columns = ['question_primary', 'follow_up']
#df_temp

In [ ]:
BAD_CHARS = ['(?) ?????']
pat = '|'.join(['({})'.format(re.escape(c)) for c in BAD_CHARS])
df_new = df_new[~df_new['question_primary'].str.contains(pat)]

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['whether'])
stop_words.extend(['whether','government','governments','fact','aware','ministry','ministries'])

In [ ]:
data = df_new.question_primary.values.tolist()
#print(data)
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=1000) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=1000)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])# if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_trigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ','VERB', 'ADV'])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
# from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# import matplotlib.pyplot as plt
# #% matplotlib inline
# from collections import Counter
# # create a word frequency dictionary
# wordfreq = Counter(id2word.values())
# # draw a Word Cloud with word frequencies
# wordcloud = WordCloud(width=900,
#                       height=500,
#                       max_words=200,
#                       max_font_size=100,
#                       relative_scaling=0.5,
#                       colormap='cubehelix_r',
#                       normalize_plurals=True).generate_from_frequencies(wordfreq)
# plt.figure(figsize=(17,14))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [ ]:
mallet_path = 'C:\mallet\mallet-2.0.8\\bin\mallet' # update this path
#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word,iterations=100,random_seed=2020)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):

    coherence_values = []
    model_list = []
    alpha_list = []
    for num_topics in range(start, limit, step):
        alpha = 50/num_topics
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary, random_seed=2020,iterations=50)#alpha=alpha)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        alpha_list.append(alpha)
    return model_list, coherence_values, alpha_list

In [ ]:
# Can take a long time to run.
model_list, coherence_values, alpha_list = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=3, limit=30, step=3)

In [ ]:
# Show graph
limit=30; start=3; step=3;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
#plt.legend(("alpha"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv, alpha_val in zip(x, coherence_values, alpha_list):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))#, " and alpha used ",alpha_val)

In [ ]:
#coherence_values[5]
optimal_model = model_list[5]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=20))

In [ ]:
## Only the aspect size '18' is the take away from this expirement. The aspect size is input seed for unsupervised ABAE